In [2]:
# import thu vien YOLO
from ultralytics import YOLO

#Load model
model = YOLO("./Models_4_yolov8n_datav5/runs/detect/train/weights/best.pt")

WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at 'C:\Users\ThanhPC\AppData\Roaming\Ultralytics\settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
# phai bat buoc cai goi C/C++ development cua visual studio
# Kiem tra torch va CUDA co phu hop?
import torch
torch.cuda.is_available()

True

In [ ]:
# results = model(source='./predict_data/IMG_0208.MOV', stream=True)  # generator of Results objects
# for r in results:
#     boxes = r.boxes  # Boxes object for bbox outputs
#     masks = r.masks  # Masks object for segment masks outputs
#     probs = r.probs  # Class probabilities for classification outputs

In [ ]:
# import random

# import cv2
# import numpy as np
# from ultralytics import YOLO

# # opening the file in read mode
# my_file = open("data.txt", "r")

# # reading the file
# data = my_file.read()

# # replacing end splitting the text | when newline ('\n') is seen.
# class_list = data.split("\n")
# my_file.close()

# # print(class_list)

# # Generate random colors for class list
# detection_colors = []
# for i in range(len(class_list)):
#     r = random.randint(0, 255)
#     g = random.randint(0, 255)
#     b = random.randint(0, 255)
#     detection_colors.append((b, g, r))

# # load a pretrained YOLOv8 model
# model = YOLO("./runs/detect/train/weights/best.pt")

# # Vals to resize video frames | small frame optimise the run    
# frame_wid = 640
# frame_hyt = 480

# # cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture("./predict_data/IMG_0208.MOV")

# if not cap.isOpened():
#     print("Cannot open camera")
#     exit()

# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     # if frame is read correctly ret is True

#     if not ret:
#         print("Can't receive frame (stream end?). Exiting ...")
#         break

#     #  resize the frame | small frame optimise the run
#     # frame = cv2.resize(frame, (frame_wid, frame_hyt))

#     # Predict on image
#     detect_params = model.predict(source=[frame], conf=0.45, save=False)

#     # Convert tensor array to numpy
#     DP = detect_params[0].cpu().numpy()
    
#     print(DP)

#     if len(DP) != 0:
#         for i in range(len(detect_params[0])):
#             print(i)

#             boxes = detect_params[0].boxes
#             box = boxes[i]  # returns one box
#             clsID = box.cls.cpu().numpy()[0]
#             conf = box.conf.cpu().numpy()[0]
#             bb = box.xyxy.cpu().numpy()[0]

#             cv2.rectangle(
#                 frame,
#                 (int(bb[0]), int(bb[1])),
#                 (int(bb[2]), int(bb[3])),
#                 detection_colors[int(clsID)],
#                 3,
#             )

#             # Display class name and confidence
#             font = cv2.FONT_HERSHEY_COMPLEX
#             cv2.putText(
#                 frame,
#                 class_list[int(clsID)] + " " + str(round(conf, 3)) + "%",
#                 (int(bb[0]), int(bb[1]) - 10),
#                 font,
#                 1,
#                 (255, 255, 255),
#                 2,
#             )

#     # Display the resulting frame
#     cv2.imshow("ObjectDetection", frame)

#     # Terminate run when "Q" pressed
#     if cv2.waitKey(1) == ord("q"):
#         break

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

In [17]:
import cv2
import os
from ultralytics import YOLO
import numpy as np

# Load the YOLOv8 model

model = YOLO("./Models_5_yolov10n/runs/detect/train/weights/best.pt")
#model = YOLO("./Models_4_yolov8n_datav5/runs/detect/train/weights/best.pt")

# Open the video file
#video_path = "./predict_data/VID_20220519_144812.mp4"
#video_path = "./predict_data/VID_20230506_163430.mp4"
video_path = "./predict_data/101.jpg"
cap = cv2.VideoCapture(video_path)

# Tạo thư mục để lưu các frame nếu chưa có
output_folder = "./detected_frames"
os.makedirs(output_folder, exist_ok=True)

frame_count = 0
confidence_threshold = 0.3 # Mức độ tin cậy cho trước

# Kernel để làm nét ảnh
sharpen_kernel = np.array([[-1, -1, -1],
                           [-1,  9, -1],
                           [-1, -1, -1]])

# Vòng lặp để xử lý các frame trong video
while cap.isOpened():
    # Đọc một frame từ video
    success, frame = cap.read()

    if success:
        # Thực hiện YOLOv8 inference trên frame
        results = model(frame)

        # Kiểm tra nếu có bất kỳ đối tượng nào được detect
        if len(results[0].boxes) > 0:
            for box in results[0].boxes:
                # Lấy độ tin cậy (confidence) của đối tượng
                confidence = box.conf[0]

                # Kiểm tra nếu độ tin cậy lớn hơn ngưỡng cho trước
                if confidence > confidence_threshold:
                    # Lấy tọa độ của bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                    # Cắt phần ảnh trong bounding box
                    detected_object = frame[y1:y2, x1:x2]

                    # # Resize phần ảnh về kích thước 640x640
                    # resized_object = cv2.resize(detected_object, (640, 640))

                    # # Làm nét phần ảnh đã được resize bằng kernel
                    # sharpened_object = cv2.filter2D(resized_object, -1, sharpen_kernel)

                    # Lưu phần ảnh đã cắt dưới dạng file ảnh .jpg
                    frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
                    cv2.imwrite(frame_filename, detected_object)

                    # Tăng số đếm frame
                    frame_count += 1

        # Visualize kết quả trên frame
        annotated_frame = results[0].plot()

        # Hiển thị frame đã được annotate
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Tăng số đếm frame
        frame_count += 1

        # Thoát khỏi vòng lặp nếu nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Thoát khỏi vòng lặp nếu đã hết video
        break

# Giải phóng bộ nhớ và đóng các cửa sổ hiển thị
cap.release()
cv2.destroyAllWindows()


0: 640x480 4 bien_sos, 49.0ms
Speed: 4.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
